In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
import os
import cv2 # For image loading (or PIL)
from google.colab import files # To allow user to upload a new image

# ----------------------------------------------------------------------
# Step 1: Mount Google Drive
# ----------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# ----------------------------------------------------------------------
# Step 2: Define Paths and Load the Trained Model
# ----------------------------------------------------------------------
# --- IMPORTANT: Adjust this path to your saved 'best_model.h5' ---
# Based on your previous structure, it should be in the 'Results' folder
# within the 'inceptionv3_classification_hybrid_case' subfolder.
model_path = '/content/drive/My Drive/Rep_Galapagos_Sea_Lion_Classification/Results/inceptionv3_classification_hybrid_case/best_model.h5'

print(f"Attempting to load model from: {model_path}")

try:
    # Load the model
    model = load_model(model_path)
    print("Model loaded successfully!")
    model.summary() # Print model summary to confirm it's loaded correctly
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure the 'model_path' is correct and the file exists.")
    # Exit or handle the error appropriately if the model can't be loaded
    exit()

# Define the input image size the model expects (InceptionV3 uses 299x299)
IMG_HEIGHT, IMG_WIDTH = 299, 299

# Define your class names in the same order as they were during training
# This is crucial for interpreting the model's output probabilities
class_names = ['Arctocephalus_galapagoensis', 'Zalophus_wollebaeki']
print(f"\nModel expects to classify into these classes: {class_names}")

# ----------------------------------------------------------------------
# Step 3: Function to Load and Preprocess a Single Image
# ----------------------------------------------------------------------
def load_and_preprocess_single_image(image_path, img_size=(IMG_HEIGHT, IMG_WIDTH)):
    try:
        # Read the image using OpenCV (or PIL)
        img = cv2.imread(image_path)
        if img is None:
            raise FileNotFoundError(f"Image not found or cannot be read: {image_path}")

        # OpenCV reads images as BGR, convert to RGB for TensorFlow/Keras models
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Resize the image
        img = cv2.resize(img, img_size)

        # Convert to float32 and add batch dimension (model expects [batch_size, height, width, channels])
        img = np.expand_dims(img, axis=0)
        img = img.astype('float32')

        # Apply InceptionV3 specific preprocessing
        img = preprocess_input(img)

        return img
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# ----------------------------------------------------------------------
# Step 4: Upload and Predict on a New Image
# ----------------------------------------------------------------------
print("\n--- Ready to predict on a new image ---")
print("Please upload an image file (JPEG or PNG).")

uploaded_files = files.upload() # This will open a file upload dialog

if uploaded_files:
    for filename in uploaded_files.keys():
        print(f"Uploaded file: {filename}")

        # Save the uploaded file to Colab's temporary /content/ directory
        temp_image_path = os.path.join('/content/', filename)
        with open(temp_image_path, 'wb') as f:
            f.write(uploaded_files[filename])

        # Preprocess the uploaded image
        processed_image = load_and_preprocess_single_image(temp_image_path)

        if processed_image is not None:
            # Make a prediction
            print("Making prediction...")
            predictions = model.predict(processed_image)

            # Get the predicted class probabilities
            predicted_probabilities = predictions[0]

            # Get the index of the highest probability
            predicted_class_idx = np.argmax(predicted_probabilities)

            # Get the predicted class name
            predicted_class_name = class_names[predicted_class_idx]

            # Get the confidence score
            confidence = predicted_probabilities[predicted_class_idx] * 100 # Convert to percentage

            print(f"\n--- Prediction Results for '{filename}' ---")
            print(f"Predicted Class: {predicted_class_name}")
            print(f"Confidence: {confidence:.2f}%")

            # Optionally, print all probabilities
            print("\nAll Class Probabilities:")
            for i, prob in enumerate(predicted_probabilities):
                print(f"  {class_names[i]}: {prob*100:.2f}%")
        else:
            print(f"Could not process image '{filename}'. Prediction aborted.")
else:
    print("No file uploaded. Prediction aborted.")

Mounted at /content/drive
Attempting to load model from: /content/drive/My Drive/Rep_Galapagos_Sea_Lion_Classification/Results/inceptionv3_classification_hybrid_case/best_model.h5


Model loaded successfully!


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 55,357,988 (211.17 MB)

 Trainable params: 33,555,202 (128.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

 Optimizer params: 2 (12.00 B)


Model expects to classify into these classes: ['Arctocephalus_galapagoensis', 'Zalophus_wollebaeki']

--- Ready to predict on a new image ---
Please upload an image file (JPEG or PNG).


Saving WhatsApp Image 2025-06-02 at 11.31.11 AM.jpeg to WhatsApp Image 2025-06-02 at 11.31.11 AM.jpeg
Uploaded file: WhatsApp Image 2025-06-02 at 11.31.11 AM.jpeg
Making prediction...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

--- Prediction Results for 'WhatsApp Image 2025-06-02 at 11.31.11 AM.jpeg' ---
Predicted Class: Zalophus_wollebaeki
Confidence: 100.00%

All Class Probabilities:
  Arctocephalus_galapagoensis: 0.00%
  Zalophus_wollebaeki: 100.00%
